In [14]:
import cupy as cp
import matplotlib.pyplot as plt

# Parámetros
N = 10
Ec = 1.0
El = cp.linspace(0.1, 10, 300)
EDelta = 1.0
phi_ext = 0.0
r_values = cp.linspace(0.1, 10, 300)

phi_ZPF = (2.0 * Ec / El) ** 0.25

def destroy(N):
    return cp.diag(cp.sqrt(cp.arange(1, N)), k=1)

N_op = 1j * (destroy(N).T - destroy(N)) / phi_ZPF[:, cp.newaxis, cp.newaxis] / 2
phi_op = (destroy(N).T + destroy(N)) * phi_ZPF[:, cp.newaxis, cp.newaxis]

delta = phi_op - phi_ext



In [15]:
ReZ = cp.cos(phi_op / 2) * cp.cos((r_values[:, cp.newaxis, cp.newaxis] * phi_op / 2)) + r_values[:, cp.newaxis, cp.newaxis] * cp.sin(phi_op / 2) * cp.sin((r_values[:, cp.newaxis, cp.newaxis] * phi_op / 2))
ImZ = -cp.cos(phi_op / 2) * cp.sin((r_values[:, cp.newaxis, cp.newaxis] * phi_op / 2)) + r_values[:, cp.newaxis, cp.newaxis] * cp.sin(phi_op / 2) * cp.cos((r_values[:, cp.newaxis, cp.newaxis] * phi_op / 2))

sigmaz = cp.array([[1, 0], [0, -1]])
sigmay = cp.array([[0, -1j], [1j, 0]])

In [16]:
N_op_expanded = cp.kron(N_op, cp.eye(2))
delta_expanded = cp.kron(delta, cp.eye(2))
ReZ_expanded = cp.kron(ReZ, cp.eye(2)).reshape(El.size, r_values.size, N, 2 * N)
ImZ_expanded = cp.kron(ImZ, cp.eye(2)).reshape(El.size, r_values.size, N, 2 * N)
H = 4 * Ec * N_op_expanded ** 2 + 0.5 * El[:, cp.newaxis, cp.newaxis] * delta_expanded ** 2 + EDelta * (ReZ_expanded @ sigmaz + ImZ_expanded @ sigmay)
# Calcula el elemento de matriz <0|n|1>
matrix_element = cp.abs(H[:, 0, N, 1]).reshape(El.size, r_values.size)
# Crear gráfica
El_grid, r_grid = cp.meshgrid(El, r_values, indexing='ij')
plt.contourf(cp.asnumpy(El_grid), cp.asnumpy(r_grid), cp.asnumpy(matrix_element), cmap='viridis')
plt.xlabel('El')
plt.ylabel('r')
plt.colorbar(label='Matrix element')
plt.show()

ValueError: cannot reshape array of size 120000 into shape 18000000